In [28]:
import pickle
import sys
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

sys.path.append(os.path.abspath("../tokenizer/"))
from tok import RegexTokenizer


with open('../tokenizer/toks_vocab_5k.pkl', 'rb') as f: #loading vocab from tokenizer
    tok_dict = pickle.load(f)


with open('../poems.txt', 'r', encoding='utf-8') as f:
    poems = f.read()

In [29]:
#creating tokenizer and adding vocab and merges trained a priori
vocab3k = dict(list(tok_dict['vocab'].items())[:3000]) #cutting the vocab size to 3k
merges3k= dict(list(tok_dict['merges'].items())[:(3000-256)])

tokenizer = RegexTokenizer()
tokenizer.vocab = vocab3k
tokenizer.merges = merges3k

#encoding the dataset
encoded_poems = tokenizer.encode(poems)

In [30]:
#defining hyperparameters

parameters = {'block_size':8, 'n_embd': 32, 'n_head':4, 'n_layer':4, 'dropout':0.2, 'vocab_size': len(tokenizer.vocab)}
batch_size = 32
block_size = parameters['block_size']
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200

In [31]:
#making it a torch tensor and splitting into training and validation sets
data = torch.tensor(encoded_poems, dtype=torch.long)
print(f'Dataset with {len(poems)} now is represented by {len(data)} tokens')
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

Dataset with 1349346 now is represented by 423242 tokens


In [32]:
#data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [33]:
from transformer import GPTLanguageModel

model = GPTLanguageModel(parameters)
m = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [34]:
#training
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    #sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 8.1673, val loss 8.1714
step 500: train loss 6.6401, val loss 6.7454
step 1000: train loss 6.2561, val loss 6.3639
step 1500: train loss 5.9364, val loss 6.1022
step 2000: train loss 5.7689, val loss 5.9374
step 2500: train loss 5.6297, val loss 5.7953
step 3000: train loss 5.4983, val loss 5.7099
step 3500: train loss 5.4087, val loss 5.5880
step 4000: train loss 5.3391, val loss 5.5241
step 4500: train loss 5.2656, val loss 5.4918


In [27]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(tokenizer.decode(m.generate(context, max_new_tokens=500)[0].tolist()))

  frubeias! Irow, and sti monte, qucer on nows on, the confuso da relig which
In no chão odiar em que não são se... Seg também, não partSoarmos ao seio.
O seu ar que interessa trabalinação
AT, o femta a sorte,

Co também dos jardins irmã.

«ire this-me umístplrom notara de gas come —
Minha alma, grional
Vós que não fiz Emgal?
É todos os meusviate para os meus versos

Dbo E levira, Há e consegue que tudo — olhando lá. O momento mais
E se deseja saber a infrvaseiro doves
Néória com um olhar de olhar. Uma mesma obra dormualidade que



es. Que há o com um liga à obra os melizar,
Da vida, o que eu conceispada tão!...,
Que se é a água à realidade na estrada,
E tudo a vontade de mim da falsa
E ele tem tão realidade não no mundo e tocado ir
(Se abpobgo não houvesse, nem bcos erTA deles,  Ah no vez que em que fui para me preito sorri
Intereé essas tirar que esse aras de que ves. Um com grandes o rierro fativo noind, do cais

Moré-me só isso? ainda melhor fisos.
Cin] que me apavoravas assim não